In [1]:
import pandas as pd
import json
import numpy as np
from operator import itemgetter


In [2]:
df= pd.read_csv('movie_metadata.csv')
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [3]:
df.shape

(5043, 28)

In [4]:
ds= df[['genres', 'movie_title', 'imdb_score', 'movie_imdb_link']].copy()

In [5]:
ds

,genres,movie_title,imdb_score,movie_imdb_link
0,Action|Adventure|Fantasy|Sci-Fi,Avatar,7.9,http://www.imdb.com/title/tt0499549/?ref_=fn_t...
1,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,7.1,http://www.imdb.com/title/tt0449088/?ref_=fn_t...
2,Action|Adventure|Thriller,Spectre,6.8,http://www.imdb.com/title/tt2379713/?ref_=fn_t...
3,Action|Thriller,The Dark Knight Rises,8.5,http://www.imdb.com/title/tt1345836/?ref_=fn_t...
4,Documentary,Star Wars: Episode VII - The Force Awakens ...,7.1,http://www.imdb.com/title/tt5289954/?ref_=fn_t...
...,...,...,...,...
5038,Comedy|Drama,Signed Sealed Delivered,7.7,http://www.imdb.com/title/tt3000844/?ref_=fn_t...
5039,Crime|Drama|Mystery|Thriller,The Following,7.5,http://www.imdb.com/title/tt2071645/?ref_=fn_t...
5040,Drama|Horror|Thriller,A Plague So Pleasant,6.3,http://www.imdb.com/title/tt2107644/?ref_=fn_t...
5041,Comedy|Drama|Romance,Shanghai Calling,6.3,http://www.imdb.com/title/tt2070597/?ref_=fn_t...


In [6]:
fetch_genres= [ds.loc[i]['genres'].split('|') for i in ds.index]

In [7]:
fetch_genres

[['Action', 'Adventure', 'Fantasy', 'Sci-Fi'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Action', 'Adventure', 'Thriller'],
 ['Action', 'Thriller'],
 ['Documentary'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Romance'],
 ['Adventure',
  'Animation',
  'Comedy',
  'Family',
  'Fantasy',
  'Musical',
  'Romance'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Adventure', 'Family', 'Fantasy', 'Mystery'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Action', 'Adventure', 'Western'],
 ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'],
 ['Action', 'Adventure', 'Family', 'Fantasy'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Action', 'Adventure', 'Comedy', 'Family', 'Fantasy', 'Sci-Fi'],
 ['Adventure', 'Fantasy'],
 ['Action', 'Adventure', 'Fantasy'],
 ['Action', 'Adventure', 'Drama', 'History'],
 ['Adventure', 'Fantasy'],
 ['Adventure', 'Family', 'Fantasy'],
 ['A

In [8]:
genres= sorted(list(set([item for sublist in fetch_genres for item in sublist])))
genres

['Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Thriller',
 'War',
 'Western']

In [9]:
data=list()
movie_title=list()

In [10]:
for i in ds.index:
    movie_title.append((ds.loc[i]['movie_title'].strip(), i, ds.loc[i]['movie_imdb_link'].strip())) #appends movie title and movie index
    movie_data = [1 if genre in ds.loc[i]['genres'].split('|') else 0 for genre in genres]
    movie_data.append(ds.loc[i]['imdb_score'])
    data.append(movie_data)
    
    

In [11]:
dump_data =r'data.json'
dump_title = r'titles.json'
with open(dump_data, 'w+', encoding='utf-8') as f:
    json.dump(data, f)
with open(dump_title, 'w+', encoding='utf-8') as f:
    json.dump(movie_title, f)
    